In [1]:
import pandas as pd
import os
import shutil
from datetime import datetime

# Function to find the newest CSV file starting with "dictionary_"
def find_newest_csv(directory):
    data_folders = os.path.join(directory, 'Data')
    csv_files = [file for file in os.listdir(data_folders) if file.startswith('dictionary_') and file.endswith('.csv')]
    if not csv_files:
        return None
    return os.path.join(data_folders, max(csv_files, key=lambda f: os.path.getctime(os.path.join(data_folders, f))))

# Function to find the newest folder starting with "fits" within the "Data" directory
def find_newest_fit_folder(directory):
    data_folders = os.path.join(directory, 'Data')
    fits_folders = [folder for folder in os.listdir(data_folders) if folder.startswith('fits') and os.path.isdir(os.path.join(data_folders, folder))]
    if not fits_folders:
        print("No folder starting with 'fits' found within the 'Data' directory.")
        return None
    return os.path.join(data_folders, max(fits_folders, key=lambda f: os.path.getctime(os.path.join(data_folders, f))))

# Find the newest CSV file
csv_file = find_newest_csv('./')
if csv_file is None:
    print("No CSV file starting with 'dictionary_' found.")
    exit()

# Load CSV file
# df = pd.read_csv(csv_file)
df = pd.read_csv("./Data/fits/dictionary_0.csv")

# Get list of existing input and output filenames and labels
existing_input_filenames = df['input'].tolist()
existing_output_filenames = df['output'].tolist()
existing_labels = df['label'].tolist()  # Retain existing labels

# Function to find the next available tic number
def find_next_tic(existing_shortnames, destination_folder):
    existing_tics = [int(output.split('.')[0][3:]) for output in existing_shortnames if output.startswith('tic')]
    new_tics = [int(output.split('.')[0][3:]) for output in os.listdir(destination_folder) if output.startswith('tic')]
    all_tics = set(range(1, max(existing_tics + new_tics, default=0) + 2))  # Add 2 to ensure we cover the next available tic
    available_tics = all_tics - set(existing_tics + new_tics)
    return min(available_tics)

# Function to copy FIT files from source_folder to dest_folder
def copy_all_fit_files(source_folder, dest_folder, existing_input_filenames, existing_labels):
    new_tic_number = find_next_tic(existing_output_filenames, dest_folder)
    for file_name in os.listdir(source_folder):
        if file_name.endswith('.fit') and file_name not in existing_input_filenames:
            # Find the next available TIC name
            while f'tic{new_tic_number}.fit' in existing_output_filenames or f'tic{new_tic_number}.fit' in os.listdir(dest_folder):
                new_tic_number += 1
            # Copy the FIT file to the destination folder
            new_tic = f'tic{new_tic_number}.fit'
            new_file_name = os.path.join(dest_folder, new_tic)
            shutil.copy(os.path.join(source_folder, file_name), new_file_name)
            # Update CSV with new tic name and label
            df.loc[len(df)] = [file_name, new_tic, 0]  # Assign label 0 for files from news_fit_folder
            existing_input_filenames.append(file_name)
            existing_output_filenames.append(new_tic)
            existing_labels.append(0)  # Retain existing labels

# Find the newest folder starting with "fits" within the "Data" directory
latest_fit_folder = find_newest_fit_folder('./')
if latest_fit_folder is None:
    print("No folder starting with 'fits' found.")
    exit()

# Directory containing FIT files from the latest fit folder
# latest_fit_folder_path = latest_fit_folder
latest_fit_folder_path = "./Data\\fits"
print(latest_fit_folder_path)

# news_fit_folder = './Data/2024-02-14/2024-02-14/2024-02-14/'
# news_fit_folder = './Data/2024-02-21/2024-02-21/2024-02-21/'

news_fit_folder = './Data/2024-02-02/2024-02-02/2024-02-02/'    #checked
# news_fit_folder = './Data/2024-02-28/2024-02-28/2024-02-28/'    #checked
# news_fit_folder = './Data/2024-03-01/2024-03-01/2024-03-01/'    #checked

if not os.path.exists(news_fit_folder):
    print("No 'news' folder found.")
    exit()

# Destination folder for copied FIT files
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
destination_folder = os.path.join('.', 'Data', f'fits_{current_datetime}/')
os.makedirs(destination_folder, exist_ok=True)  # Create the destination folder

# Copy existing FIT files to the new folder
for idx, label in enumerate(existing_labels):
    file_name = existing_output_filenames[idx]
    if file_name.startswith('tic'):
        if file_name not in os.listdir(destination_folder):
            shutil.copy(os.path.join(latest_fit_folder_path, file_name), os.path.join(destination_folder, file_name))
    elif label == 0:
        if file_name not in os.listdir(destination_folder):
            shutil.copy(os.path.join(latest_fit_folder_path, file_name), os.path.join(destination_folder, file_name))

# Call the function to copy all FIT files from the news fit folder
copy_all_fit_files(news_fit_folder, destination_folder, existing_input_filenames, existing_labels)

# Save updated dictionary CSV with current datetime in name
new_dictionary_csv = os.path.join('.', 'Data', f'dictionary_{current_datetime}.csv')
df.to_csv(new_dictionary_csv, index=False)

print("Updated dictionary CSV file created in Data folder:", new_dictionary_csv)


./Data\fits
Updated dictionary CSV file created in Data folder: .\Data\dictionary_2024-04-11_14-10-55.csv
